# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [3]:
token = getpass()


········


In [9]:
url = 'https://api.foursquare.com/v3/places/search?query=coffee&ll=41.3977421%2C2.190249315&limit=10'
headers = {
    "accept": "application/json",
    "Authorization": token
}
res = requests.get(url, headers=headers)
res

<Response [200]>

In [10]:
res.json()['results'][0]



{'fsq_id': '555f2357498ee6528074710b',
 'categories': [{'id': 13034,
   'name': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
    'suffix': '.png'}},
  {'id': 13035,
   'name': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}},
  {'id': 13065,
   'name': 'Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 409,
 'geocodes': {'main': {'latitude': 41.398735, 'longitude': 2.195069},
  'roof': {'latitude': 41.398735, 'longitude': 2.195069}},
 'link': '/v3/places/555f2357498ee6528074710b',
 'location': {'address': 'Carrer Badajoz, 95',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': '',
  'formatted_address': 'Carrer Badajoz, 95, 08005 Barcelona Catalonia',
  'locality': 'Barcelona',
  'postcode': '08005',
  'region': 'Catalonia'},
 'name': 'Espai Joliu',
 'related_places': {},

In [12]:
len(res.json()['results'][0])

10

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [14]:
url = 'https://api.foursquare.com/v3/places/search?query=coffee&radius=500&limit=5' 

res = requests.get(url, headers=headers)
res

<Response [200]>

In [15]:
res.json()['results'][0]


{'fsq_id': '4cb5a6401b0af04d9384ca25',
 'categories': [{'id': 13035,
   'name': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}}],
 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
   'name': 'Starbucks'}],
 'distance': 101,
 'geocodes': {'main': {'latitude': 41.387848, 'longitude': 2.169634}},
 'link': '/v3/places/4cb5a6401b0af04d9384ca25',
 'location': {'admin_region': 'Cataluña',
  'country': 'ES',
  'formatted_address': 'Barcelona Catalonia',
  'locality': 'Barcelona',
  'region': 'Catalonia'},
 'name': 'Starbucks',
 'related_places': {},
 'timezone': 'Europe/Madrid'}

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [31]:
def foursquare_places (venue, coordinates):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates[0]}%2C{coordinates[1]}&limit=5"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers)
    full_response = response.json()
    location_data = pd.DataFrame(full_response["results"])
    
    return location_data[['name','location']]

In [35]:
foursquare_places ('coffee shops', [40.4169376,-3.706187217])

,name,location
0,Toma Café,"{'address': 'Calle la Palma, 49', 'admin_regio..."
1,HanSo Café,"{'address': 'Calle del Pez, 20', 'admin_region..."
2,Hola Coffee,"{'address': 'Calle Doctor Fourquet, 33', 'admi..."
3,Ruda Café,"{'address': 'Calle Ruda, 11', 'admin_region': ..."
4,Café de la Luz,"{'address': 'Calle Puebla, 8', 'admin_region':..."


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [36]:
coffee_shops = foursquare_places ('coffee shops', [40.4169376,-3.706187217])

In [42]:
! mkdir data

In [43]:
json_ = coffee_shops.to_json('data/places.json', orient="records")
json_

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [47]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['ironcoffee']
c = db.get_collection('ironcoffee')
c.find_one()

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 63e51bffe3b020f9f5a0d655, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>